In [1]:
import os
import time

import numpy as np

from birddata import data
import constants
import preprocessing
from top_birds import get_reduced_scores

2024-03-20 15:09:17.640853: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-20 15:09:17.640902: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-20 15:09:17.641789: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-20 15:09:17.648140: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 15:09:18.465498: W tensorflow/compiler/tf2

In [2]:
metadata = data.Data(os.path.join(constants.base_dir, constants.metadata_csv))
fold_dict = metadata.get_folds()

NUM_CLASSES 264


/mnt/efs/birdclef/birddata/data.py:53: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_df = train_df[keep_rows]


In [3]:
def do_scores(row):
    # file = "../train_audio_test/"+row.filename.split('.')[0]+".mp3"
    file = preprocessing.ogg_fn(row.filename)
    try:
        score = get_reduced_scores(file).numpy()
    except Exception as e:
        print(f"Exception {row.name} {file}:" + str(e))
        score = np.zeros((264,))
    print(file, row.name, score.shape, score[row.primary_index])
    return score

In [4]:
now = time.time()

In [5]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

# fold_dict['valid'] = fold_dict['valid'].sample(frac=1.)
scores = fold_dict['valid'].apply(do_scores, axis=1)
fold_dict['valid'].loc[:, ('scores',)] = scores
fold_dict['valid'].to_pickle('scores.pkl')

2024-03-20 15:10:18.654861: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907


./birdsong-recognition/train_audio/aldfly/XC142065.ogg 12 (264,) 10.934933
./birdsong-recognition/train_audio/aldfly/XC142066.ogg 13 (264,) 3.2114282
./birdsong-recognition/train_audio/aldfly/XC142067.ogg 14 (264,) 10.870186
./birdsong-recognition/train_audio/aldfly/XC142068.ogg 15 (264,) 9.784571
./birdsong-recognition/train_audio/aldfly/XC180091.ogg 31 (264,) 12.989799
./birdsong-recognition/train_audio/aldfly/XC189042.ogg 38 (264,) 9.084529
./birdsong-recognition/train_audio/aldfly/XC189043.ogg 39 (264,) 14.175631
./birdsong-recognition/train_audio/aldfly/XC189044.ogg 40 (264,) 7.8577304
./birdsong-recognition/train_audio/aldfly/XC189045.ogg 41 (264,) 12.296926
./birdsong-recognition/train_audio/aldfly/XC194087.ogg 54 (264,) 13.673862
./birdsong-recognition/train_audio/aldfly/XC194088.ogg 55 (264,) 13.88627
./birdsong-recognition/train_audio/aldfly/XC269063.ogg 67 (264,) 11.268232
./birdsong-recognition/train_audio/aldfly/XC31060.ogg 72 (264,) 10.496399
./birdsong-recognition/train_

In [6]:
time.time()-now

188.75067281723022